In [15]:
import torch
import time
import numpy as np
import gymnasium as gym
import flappy_bird_gymnasium
import torch
from model import DQN_CNN
from save_model.utils import preprocess_frame, transition
from gymnasium.wrappers import RecordVideo
from collections import deque
import os

def play_flappy_bird_live(
    policy_net,
    env,
    device,
    frame_skip=4,
    fps=30
):
    """
    Plays a live game of Flappy Bird using a trained DQN model.

    Args:
        policy_net (torch.nn.Module): Loaded DQN model
        env (gym.Env): Flappy Bird gym environment
        device (torch.device)
        frame_skip (int): Number of frames the model was trained on
        fps (int): Render FPS
    """

    policy_net.eval()

    obs = env.reset()
    done = False
    total_reward = 0
    clock_delay = 1.0 / fps

    state, _, _ = transition(0, env, frame_skip)

    while not done:
        start_time = time.time()
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

        with torch.no_grad():
            q_values = policy_net(state)
            action = q_values.argmax(dim=1).item()

        next_state, reward, done = transition(action, env, frame_skip)
        total_reward += reward
        state = next_state

        # FPS control
        elapsed = time.time() - start_time
        if elapsed < clock_delay:
            time.sleep(clock_delay - elapsed)

    env.close()
    print(f"🏁 Game over | Total reward: {total_reward}")


In [16]:
def record_trained_agent_video(
    model_path="checkpoints/flappy_dqn.pt",
    video_dir="videos",
    env_name="FlappyBird-v0",
    frame_skip=4,
    device="cpu",
):
    os.makedirs(video_dir, exist_ok=True)

    # ---------- environment with video recording ----------
    env = gym.make(env_name, render_mode="rgb_array")
    env = RecordVideo(
        env,
        video_folder=video_dir,
        episode_trigger=lambda episode_id: True,  # record first episode
        name_prefix="flappy_dqn",
    )

    # ---------- load model ----------
    policy_net = DQN_CNN(frame_skip).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    policy_net.load_state_dict(checkpoint)
    policy_net.eval()

    # ---------- reset ----------
    env.reset()
    frame_stack = deque(maxlen=frame_skip)

    done = False
    total_reward = 0

    # ---------- play episode ----------
    current_state, _, _ = transition(0, env, frame_skip)

    while not done:
        with torch.no_grad():
            state_t = torch.tensor(current_state, dtype=torch.float32).unsqueeze(0).to(device)
            action = policy_net(state_t).argmax(dim=1).item()

        next_state, reward_sum, done = transition(action, env, frame_skip)
        current_state = next_state

        total_reward += reward_sum

    env.close()
    print(f"🎥 Video saved in '{video_dir}/' | Reward: {total_reward:.2f}")

In [17]:


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FRAME_SKIP = 4

record_trained_agent_video(
    model_path="best_dqn.pt",
    video_dir="videos",
    frame_skip=FRAME_SKIP,
    device=DEVICE,
)




/Users/stefaniancu/Documents/Proiecte/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /Users/stefaniancu/Documents/Proiecte/Q-learning-CNN-in-Flappy-Bird-Environment/src/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/Users/stefaniancu/Documents/Proiecte/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/stefaniancu/Documents/Proiecte/Q-learning-CNN-in-Flappy-Bird-Environment/.venv/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"

🎥 Video saved in 'videos/' | Reward: 785.00
